In [3]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(tf.__version__)
print(tf.executing_eagerly())
print(hub.__version__)
print(tf.config.experimental.list_physical_devices("GPU"))

2.2.0
True
0.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                   split=("train[:60%]", "train[60%:]", "test"),
                                                   as_supervised=True)

In [14]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [25]:
train_examples_batch[0:3].numpy()

array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot 

In [26]:
train_labels_batch[0:3].numpy()

array([0, 0, 0], dtype=int64)

In [28]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [29]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [33]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 2s 71ms/step - loss: 0.8571 - accuracy: 0.5325 - val_loss: 0.7424 - val_accuracy: 0.5790
Epoch 2/20
30/30 [==============================] - 2s 64ms/step - loss: 0.6889 - accuracy: 0.6136 - val_loss: 0.6418 - val_accuracy: 0.6410
Epoch 3/20
30/30 [==============================] - 2s 64ms/step - loss: 0.6093 - accuracy: 0.6623 - val_loss: 0.5865 - val_accuracy: 0.6842
Epoch 4/20
30/30 [==============================] - 2s 65ms/step - loss: 0.5563 - accuracy: 0.6996 - val_loss: 0.5451 - val_accuracy: 0.7109
Epoch 5/20
30/30 [==============================] - 2s 64ms/step - loss: 0.5157 - accuracy: 0.7309 - val_loss: 0.5111 - val_accuracy: 0.7409
Epoch 6/20
30/30 [==============================] - 2s 64ms/step - loss: 0.4785 - accuracy: 0.7585 - val_loss: 0.4812 - val_accuracy: 0.7687
Epoch 7/20
30/30 [==============================] - 2s 66ms/step - loss: 0.4405 - accuracy: 0.7876 - val_loss: 0.4532 - val_accuracy: 0.7673
Epoch 8/20
30

In [34]:
results = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.3150 - accuracy: 0.8593


In [36]:
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.315
accuracy: 0.859
